In [ ]:
import pandas as pd
import numpy as np

import stapes

In [ ]:
example_tri = pd.read_csv("examples/example_triangles.csv")
example_tri["DevLag"] = example_tri["DevLagId"]
example_train_tri = example_tri[example_tri["CalendarId"] <= 10]
example_test_tri = example_tri[example_tri["CalendarId"] > 10]

In [ ]:
example_train_tri

In [ ]:
train_data = {
    (name, int(row["TriangleId"]), int(row["ExpPeriodId"]), int(row["DevLagId"])): row[name]
    for name in ["ReportedLoss", "PaidLoss", "EarnedPremium", "DevLag"]
    for _, row in example_train_tri.iterrows()
    if not np.isnan(row[name])
}

In [ ]:
test_data = {
    (name, int(row["TriangleId"]), int(row["ExpPeriodId"]), int(row["DevLagId"])): row[name]
    for name in ["EarnedPremium", "DevLag"]
    for _, row in example_test_tri.iterrows()
}
test_coords = [
    ("ReportedLoss", tri_id, exp_id, dev_id)
    for name, tri_id, exp_id, dev_id in test_data
    if name == "EarnedPremium"
]

In [ ]:
model = stapes.build_model("""
pos :ata = vector(group=TriangleDevLagId);
real :noise_slope = factor(group=TriangleId, is_centered=false);
real :noise_intercept = factor(group=TriangleId, is_centered=false);
pos :power = scalar();

ReportedLoss mean = ReportedLoss.prev_dev * :ata;
ReportedLoss variance = exp(
    (:noise_slope * DevLag + :noise_intercept)
) * ReportedLoss.prev_dev ^ :power;
""")

In [ ]:
stan_data = stapes.build_stan_data(train_data, model.offsets)

In [ ]:
config = {
    "ReportedLoss__family": "gamma",
    "ata__loc": 0.0,
    "ata__scale": 3.0,
    "power__scale": 0.5,
    "noise_intercept__mu_loc": -1.0,
    "noise_intercept__mu_scale": 1.0,
    "noise_intercept__sigma_scale": 0.5,
    "ReportedLoss__missing_loc": 0.0,
    "ReportedLoss__missing_scale": 2.0,
}

In [ ]:
model.fit(train_data, config)